# Download official SAM2 weights and modeling code

In [3]:
!wget "https://dl.fbaipublicfiles.com/segment_anything_2/072824/sam2_hiera_tiny.pt"

--2025-01-14 23:00:34--  https://dl.fbaipublicfiles.com/segment_anything_2/072824/sam2_hiera_tiny.pt
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 18.154.161.79, 18.154.161.30, 18.154.161.85, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|18.154.161.79|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 155906050 (149M) [application/vnd.snesdev-page-table]
Saving to: ‘sam2_hiera_tiny.pt’

sam2_hiera_tiny.pt  100%[===================>] 148.68M  5.61MB/s    in 25s     

2025-01-14 23:00:59 (5.97 MB/s) - ‘sam2_hiera_tiny.pt’ saved [155906050/155906050]



# Install dependency with uv

In [1]:
!uv --version
!uv venv
!uv sync

uv 0.5.6 (b70c4f30e 2024-12-03)
Using CPython 3.11.11
Creating virtual environment at: .venv
Activate with: source .venv/bin/activate
Resolved 81 packages in 5ms
Installed 61 packages in 453ms                              
 + antlr4-python3-runtime==4.9.3
 + appnope==0.1.4
 + asttokens==3.0.0
 + coloredlogs==15.0.1
 + comm==0.2.2
 + debugpy==1.8.11
 + decorator==5.1.1
 + executing==2.1.0
 + filelock==3.16.1
 + flatbuffers==24.12.23
 + fsspec==2024.12.0
 + humanfriendly==10.0
 + hydra-core==1.3.2
 + iopath==0.1.10
 + ipykernel==6.29.5
 + ipython==8.31.0
 + jedi==0.19.2
 + jinja2==3.1.5
 + jupyter-client==8.6.3
 + jupyter-core==5.7.2
 + markdown-it-py==3.0.0
 + markupsafe==3.0.2
 + matplotlib-inline==0.1.7
 + mdurl==0.1.2
 + ml-dtypes==0.5.1
 + mpmath==1.3.0
 + nest-asyncio==1.6.0
 + networkx==3.4.2
 + numpy==2.2.1
 + omegaconf==2.3.0
 + onnx==1.17.0
 + onnxruntime==1.20.1
 + onnxscript==0.1.0.dev20250114
 + onnxsim==0.4.36
 + packaging==24.2
 + parso==0.8.4
 + pexpect==4.9.0
 + pillow==

# Split model into Encoder and Decoder

In [3]:
from typing import Optional, Tuple, Any
import torch
from torch import nn
import torch.nn.functional as F
from torch.nn.init import trunc_normal_

from sam2.modeling.sam2_base import SAM2Base

class SAM2ImageEncoder(nn.Module):
    def __init__(self, sam_model: SAM2Base) -> None:
        super().__init__()
        self.model = sam_model
        self.image_encoder = sam_model.image_encoder
        self.no_mem_embed = sam_model.no_mem_embed

    def forward(self, x: torch.Tensor) -> tuple[Any, Any, Any]:
        backbone_out = self.image_encoder(x)
        backbone_out["backbone_fpn"][0] = self.model.sam_mask_decoder.conv_s0(
            backbone_out["backbone_fpn"][0]
        )
        backbone_out["backbone_fpn"][1] = self.model.sam_mask_decoder.conv_s1(
            backbone_out["backbone_fpn"][1]
        )

        feature_maps = backbone_out["backbone_fpn"][-self.model.num_feature_levels:]
        vision_pos_embeds = backbone_out["vision_pos_enc"][-self.model.num_feature_levels:]

        feat_sizes = [(x.shape[-2], x.shape[-1]) for x in vision_pos_embeds]

        # flatten NxCxHxW to HWxNxC
        vision_feats = [x.flatten(2).permute(2, 0, 1) for x in feature_maps]
        vision_pos_embeds = [x.flatten(2).permute(2, 0, 1) for x in vision_pos_embeds]

        vision_feats[-1] = vision_feats[-1] + self.no_mem_embed

        feats = [feat.permute(1, 2, 0).reshape(1, -1, *feat_size)
                 for feat, feat_size in zip(vision_feats[::-1], feat_sizes[::-1])][::-1]

        return feats[0], feats[1], feats[2]


class SAM2ImageDecoder(nn.Module):
    def __init__(
            self,
            sam_model: SAM2Base,
            multimask_output: bool
    ) -> None:
        super().__init__()
        self.mask_decoder = sam_model.sam_mask_decoder
        self.prompt_encoder = sam_model.sam_prompt_encoder
        self.model = sam_model
        self.multimask_output = multimask_output

    @torch.no_grad()
    def forward(
            self,
            image_embed: torch.Tensor,
            high_res_feats_0: torch.Tensor,
            high_res_feats_1: torch.Tensor,
            point_coords: torch.Tensor,
            point_labels: torch.Tensor,
            mask_input: torch.Tensor,
            has_mask_input: torch.Tensor,
            img_size: torch.Tensor
    ):
        sparse_embedding = self._embed_points(point_coords, point_labels)
        self.sparse_embedding = sparse_embedding
        dense_embedding = self._embed_masks(mask_input, has_mask_input)

        high_res_feats = [high_res_feats_0, high_res_feats_1]
        image_embed = image_embed

        masks, iou_predictions, _, _ = self.mask_decoder.predict_masks(
            image_embeddings=image_embed,
            image_pe=self.prompt_encoder.get_dense_pe(),
            sparse_prompt_embeddings=sparse_embedding,
            dense_prompt_embeddings=dense_embedding,
            repeat_image=False,
            high_res_features=high_res_feats,
        )

        if self.multimask_output:
            masks = masks[:, 1:, :, :]
            iou_predictions = iou_predictions[:, 1:]
        else:
            masks, iou_predictions = self.mask_decoder._dynamic_multimask_via_stability(masks, iou_predictions)

        masks = torch.clamp(masks, -32.0, 32.0)
        print(masks.shape, iou_predictions.shape)

        masks = F.interpolate(masks, (img_size[0], img_size[1]), mode="bilinear", align_corners=False)

        return masks, iou_predictions

    def _embed_points(self, point_coords: torch.Tensor, point_labels: torch.Tensor) -> torch.Tensor:

        point_coords = point_coords + 0.5

        padding_point = torch.zeros((point_coords.shape[0], 1, 2), device=point_coords.device)
        padding_label = -torch.ones((point_labels.shape[0], 1), device=point_labels.device)
        point_coords = torch.cat([point_coords, padding_point], dim=1)
        point_labels = torch.cat([point_labels, padding_label], dim=1)

        point_coords[:, :, 0] = point_coords[:, :, 0] / self.model.image_size
        point_coords[:, :, 1] = point_coords[:, :, 1] / self.model.image_size

        point_embedding = self.prompt_encoder.pe_layer._pe_encoding(point_coords)
        point_labels = point_labels.unsqueeze(-1).expand_as(point_embedding)

        point_embedding = point_embedding * (point_labels != -1)
        point_embedding = point_embedding + self.prompt_encoder.not_a_point_embed.weight * (
                point_labels == -1
        )

        for i in range(self.prompt_encoder.num_point_embeddings):
            point_embedding = point_embedding + self.prompt_encoder.point_embeddings[i].weight * (point_labels == i)

        return point_embedding

    def _embed_masks(self, input_mask: torch.Tensor, has_mask_input: torch.Tensor) -> torch.Tensor:
        mask_embedding = has_mask_input * self.prompt_encoder.mask_downscaling(input_mask)
        mask_embedding = mask_embedding + (
                1 - has_mask_input
        ) * self.prompt_encoder.no_mask_embed.weight.reshape(1, -1, 1, 1)
        return mask_embedding

# Convert model to .onnx

## Encoder

In [4]:
import torch
from sam2.build_sam import build_sam2

model_type = "sam2_hiera_tiny"
model_cfg = "sam2_hiera_t.yaml"
input_size = 1024 
multimask_output = True

sam2_checkpoint = f"./{model_type}.pt"
sam2_model = build_sam2(model_cfg, sam2_checkpoint, device="cpu")

img=torch.randn(1, 3, input_size, input_size).cpu()

sam2_encoder = SAM2ImageEncoder(sam2_model).cpu()
high_res_feats_0, high_res_feats_1, image_embed = sam2_encoder(img)
print(high_res_feats_0.shape)
print(high_res_feats_1.shape)
print(image_embed.shape)

torch.onnx.export(sam2_encoder,
      img,
      f"{model_type}_encoder.onnx",
      export_params=True,
      opset_version=17,
      do_constant_folding=True,
      input_names = ['image'],
      output_names = ['high_res_feats_0', 'high_res_feats_1', 'image_embed'],
    )

torch.Size([1, 32, 256, 256])
torch.Size([1, 64, 128, 128])
torch.Size([1, 256, 64, 64])


/Users/giuseppeambrosio/Desktop/projects/vite-sam/notebooks/.venv/lib/python3.11/site-packages/sam2/modeling/backbones/utils.py:30: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if pad_h > 0 or pad_w > 0:
/Users/giuseppeambrosio/Desktop/projects/vite-sam/notebooks/.venv/lib/python3.11/site-packages/sam2/modeling/backbones/utils.py:58: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if Hp > H or Wp > W:


## Decoder

In [5]:
sam2_decoder = SAM2ImageDecoder(sam2_model, multimask_output=multimask_output).cpu()

embed_dim = sam2_model.sam_prompt_encoder.embed_dim
embed_size = (sam2_model.image_size // sam2_model.backbone_stride, sam2_model.image_size // sam2_model.backbone_stride)
mask_input_size = [4 * x for x in embed_size]
print(embed_dim, embed_size, mask_input_size)

point_coords = torch.randint(low=0, high=input_size, size=(1, 5, 2), dtype=torch.float)
point_labels = torch.randint(low=0, high=1, size=(1, 5), dtype=torch.float)
mask_input = torch.randn(1, 1, *mask_input_size, dtype=torch.float)
has_mask_input = torch.tensor([1], dtype=torch.float)
orig_im_size = torch.tensor([input_size, input_size], dtype=torch.int32)

masks, scores = sam2_decoder(image_embed, high_res_feats_0, high_res_feats_1, point_coords, point_labels, mask_input, has_mask_input, orig_im_size)

torch.onnx.export(sam2_decoder,
      (image_embed, high_res_feats_0, high_res_feats_1, point_coords, point_labels, mask_input, has_mask_input, orig_im_size),
      f"{model_type}_decoder.onnx",
      export_params=True,
      opset_version=16,
      do_constant_folding=True,
      input_names = ['image_embed', 'high_res_feats_0', 'high_res_feats_1', 'point_coords', 'point_labels', 'mask_input', 'has_mask_input', 'orig_im_size'],
      output_names = ['masks', 'iou_predictions'],
      dynamic_axes = {"point_coords": {0: "num_labels", 1: "num_points"},
                      "point_labels": {0: "num_labels", 1: "num_points"},
                      "mask_input": {0: "num_labels"},
                      "has_mask_input": {0: "num_labels"}
      }
    )


256 (64, 64) [256, 256]
torch.Size([1, 3, 256, 256]) torch.Size([1, 3])


/Users/giuseppeambrosio/Desktop/projects/vite-sam/notebooks/.venv/lib/python3.11/site-packages/sam2/modeling/sam/mask_decoder.py:203: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  assert image_embeddings.shape[0] == tokens.shape[0]
/Users/giuseppeambrosio/Desktop/projects/vite-sam/notebooks/.venv/lib/python3.11/site-packages/sam2/modeling/sam/mask_decoder.py:207: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  image_pe.size(0) == 1


torch.Size([1, 3, 256, 256]) torch.Size([1, 3])


# Test exported models with `onnxruntime`

## Encoder

In [6]:
import onnx

onnx_model = onnx.load("sam2_hiera_tiny_encoder.onnx")
onnx.checker.check_model(onnx_model)

In [7]:
import onnxruntime as ort
import numpy as np

ort_sess = ort.InferenceSession('sam2_hiera_tiny_encoder.onnx', {})

2025-01-14 23:24:42.614163 [W:onnxruntime:, graph.cc:109 MergeShapeInfo] Error merging shape info for output. '/image_encoder/trunk/Concat_3_output_0' source:{4} target:{5}. Falling back to lenient merge.


In [8]:
img = torch.randn(1, 3, input_size, input_size).cpu()

outputs = ort_sess.run(None, {
    'image': img.numpy(),
})

for i in range(len(outputs)):
    print (ort_sess.get_outputs()[i].name)
    print (outputs[i].shape)

high_res_feats_0
(1, 32, 256, 256)
high_res_feats_1
(1, 64, 128, 128)
image_embed
(1, 256, 64, 64)


## Decoder

In [9]:
onnx_model = onnx.load("sam2_hiera_tiny_decoder.onnx")
onnx.checker.check_model(onnx_model)

In [10]:
ort_sess = ort.InferenceSession('sam2_hiera_tiny_decoder.onnx', {})

outputs = ort_sess.run(None, {
    'image_embed': image_embed.detach().numpy(), 
    'high_res_feats_0': high_res_feats_0.detach().numpy(), 
    'high_res_feats_1': high_res_feats_1.detach().numpy(), 
    'point_coords': point_coords.numpy(), 
    'point_labels': point_labels.numpy(), 
    'mask_input': mask_input.numpy(), 
    'has_mask_input': has_mask_input.numpy(), 
    'orig_im_size': orig_im_size.numpy()
})

for i in range(len(outputs)):
    print (ort_sess.get_outputs()[i].name)
    print (outputs[i].shape)

masks
(1, 3, 1024, 1024)
iou_predictions
(1, 3)


# Convert Encoder model .onnx to .ort

In [11]:
!python3 -m onnxruntime.tools.convert_onnx_models_to_ort sam2_hiera_tiny_encoder.onnx

Converting models with optimization style 'Fixed' and level 'all'
Converting optimized ONNX model /Users/giuseppeambrosio/Desktop/projects/vite-sam/notebooks/sam2_hiera_tiny_encoder.onnx to ORT format model /Users/giuseppeambrosio/Desktop/projects/vite-sam/notebooks/sam2_hiera_tiny_encoder.ort
2025-01-14 23:25:25.476227 [W:onnxruntime:, graph.cc:109 MergeShapeInfo] Error merging shape info for output. '/image_encoder/trunk/Concat_3_output_0' source:{4} target:{5}. Falling back to lenient merge.
Converted 1/1 models successfully.
Generating config file from ORT format models with optimization style 'Fixed' and level 'all'
2025-01-14 23:25:26,213 ort_format_model.utils [INFO] - Created config in /Users/giuseppeambrosio/Desktop/projects/vite-sam/notebooks/sam2_hiera_tiny_encoder.required_operators.config
Converting models with optimization style 'Runtime' and level 'all'
Converting optimized ONNX model /Users/giuseppeambrosio/Desktop/projects/vite-sam/notebooks/sam2_hiera_tiny_encoder.onn